# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [47]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test]).reset_index()
df.drop(['index'], axis=1, inplace=True)

In [48]:
object_features = list(df.select_dtypes(include=['object']).columns)
numeric_features = list(df.select_dtypes(include=['int64', 'float64']).columns)

train_num = train_Y.shape[0]

df_obj = df[object_features].fillna('None')
df_num = df[numeric_features].fillna(df.mean())

# 作業2
* 承上題，三者比較效果何者最好?
* ans : 計數編碼較好

In [49]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df_obj.columns:
    df_temp[c] = LabelEncoder().fit_transform(df_obj[c])
df_revised = pd.concat([df_temp, df_num], axis=1)
train_X = df_revised[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7890443143400547


In [50]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df = pd.merge(df, count_df, on='Cabin', how='left').fillna(0)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [51]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df_obj[col])
df_temp['Cabin_Count'] = df['Cabin_Count']
df_revised = pd.concat([df_temp, df_num], axis=1)
train_X = df_revised[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.8013915235441542


In [53]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df_obj[col])
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
df_revised = pd.concat([df_temp, df_num], axis=1)
train_X = df_revised[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X , train_Y, cv=5).mean())

0.7912663970718355


In [54]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df_obj[col])

df_temp['Cabin_Count'] = df['Cabin_Count']
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
df_revised = pd.concat([df_temp, df_num], axis=1)
train_X = df_revised[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X , train_Y, cv=5).mean())

0.7957609209489936
